## Modules & Settings

In [1]:
import os
import sys
sys.path.append('..')

from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from bsoption.nyopchain import NYopchain
Opchain = NYopchain()

from litedata import *
from litedata.loadyahoodata import getyahoodata

## Option chain query examples

### single expiry, two adjacent trading days, only put, order by strike then tradedate

In [2]:
inputdict1 = {'asset': 'NVDA',  
              'optype': ('P', 'P'), # Option type: ("C", "C") = CALL only, ("P", "P") = PUT only, ("C", "P") = CALL & PUT
              'strike_lowerbound': 150,  # Lower bound for range of strike prices (inclusive)
              'strike_upperbound': 180,  # Upper bound for range of strike prices (inclusive)
              'startexpiry': '2022-08-27',  # query expiry startdate (inclusive)
              'endexpiry': '2022-09-02',  # query expiry enddate (inclusive)              
              'starttd': '2022-08-25',  # query period startdate (inclusive)
              'endtd': '2022-08-26'  # query period startdate (inclusive) 
             }

orderfield1 = ['optype', 'strike', 'tradedate']
dfop1 = Opchain.loadopdata(inputdict1, orderfield=orderfield1)
dfop1

,tradedate,asset,optype,expiry,strike,iv,vol,oi,last,bid,ask,mid
0,2022-08-25,NVDA,P,2022-09-02,150.0,62.89,3692.0,3178,0.17,0.15,0.18,0.165
1,2022-08-26,NVDA,P,2022-09-02,150.0,57.96,10752.0,3246,1.05,1.02,1.05,1.035
2,2022-08-25,NVDA,P,2022-09-02,152.5,61.13,685.0,1050,0.23,0.21,0.24,0.225
3,2022-08-26,NVDA,P,2022-09-02,152.5,57.30,3013.0,1337,1.48,1.47,1.49,1.480
4,2022-08-25,NVDA,P,2022-09-02,155.0,59.38,1897.0,1831,0.30,0.29,0.32,0.305
5,2022-08-26,NVDA,P,2022-09-02,155.0,56.57,5881.0,1954,2.07,2.05,2.07,2.060
6,2022-08-25,NVDA,P,2022-09-02,157.5,57.72,1360.0,1108,0.42,0.40,0.43,0.415
7,2022-08-26,NVDA,P,2022-09-02,157.5,55.71,4499.0,1299,2.77,2.77,2.81,2.790
8,2022-08-25,NVDA,P,2022-09-02,160.0,56.06,8675.0,3841,0.56,0.55,0.57,0.560
9,2022-08-26,NVDA,P,2022-09-02,160.0,54.88,26134.0,6545,3.73,3.65,3.75,3.700


### Example 2:  Single option contract across days

In [3]:
inputdict2 = {'asset': 'QQQ',  
              'optype': ('P', 'P'), # Option type: ("C", "C") = CALL only, ("P", "P") = PUT only, ("C", "P") = CALL & PUT
              'strike_lowerbound': 270,  # Lower bound for range of strike prices (inclusive)
              'strike_upperbound': 270,  # Upper bound for range of strike prices (inclusive)
              'startexpiry': '2022-10-14',  # query expiry startdate (inclusive)
              'endexpiry': '2022-10-14',  # query expiry enddate (inclusive)              
              'starttd': '2022-10-01',  # query period startdate (inclusive)
              'endtd': '2022-10-14'  # query period startdate (inclusive) 
             }

dfop2 = Opchain.loadopdata(inputdict2)
dfop2

,tradedate,asset,optype,expiry,strike,iv,vol,oi,last,bid,ask,mid
0,2022-10-03,QQQ,P,2022-10-14,270.0,35.38,34892.0,33224,5.01,4.94,5.04,4.990
1,2022-10-04,QQQ,P,2022-10-14,270.0,37.33,12617.0,4246,2.38,2.36,2.39,2.375
2,2022-10-05,QQQ,P,2022-10-14,270.0,36.52,13483.0,11539,2.05,2.03,2.05,2.040
3,2022-10-06,QQQ,P,2022-10-14,270.0,38.92,10090.0,18672,2.60,2.56,2.60,2.580
4,2022-10-07,QQQ,P,2022-10-14,270.0,37.27,21365.0,18806,5.94,5.85,6.01,5.930
5,2022-10-10,QQQ,P,2022-10-14,270.0,43.73,6406.0,14919,6.72,6.76,6.90,6.830
6,2022-10-11,QQQ,P,2022-10-14,270.0,45.51,5813.0,13802,8.73,8.64,8.93,8.785
7,2022-10-12,QQQ,P,2022-10-14,270.0,54.03,2445.0,0,8.79,8.81,8.97,8.890
8,2022-10-13,QQQ,P,2022-10-14,270.0,51.05,18217.0,11737,3.36,3.20,3.50,3.350


### Last trading day's option chain with strike in a price range

In [4]:
ltdstr = getltdate(holidaydictny)
ltddate = datetime.strptime(ltdstr, '%Y-%m-%d')
nextexpiry = (ltddate + timedelta(days=7)).strftime('%Y-%m-%d')
next2expiry = (ltddate + timedelta(days=13)).strftime('%Y-%m-%d')

inputdict3 = {'asset': 'AAPL',  
              'optype': ('C', 'P'), # Option type: ("C", "C") = CALL only, ("P", "P") = PUT only, ("C", "P") = CALL & PUT
              'strike_lowerbound': 125,  # Lower bound for range of strike prices (inclusive)
              'strike_upperbound': 165,  # Upper bound for range of strike prices (inclusive)
              'startexpiry': nextexpiry,  # query expiry startdate (inclusive)
              'endexpiry': next2expiry,  # query expiry enddate (inclusive)              
              'starttd': ltdstr,  # query period startdate (inclusive)
              'endtd': ltdstr  # query period startdate (inclusive) 
             }

dfop3 = Opchain.loadopdata(inputdict3)
dfop3

,tradedate,asset,optype,expiry,strike,iv,vol,oi,last,bid,ask,mid
0,2022-11-11,AAPL,C,2022-11-18,125.0,53.52,45.0,3025.0,24.70,24.50,24.95,24.725
1,2022-11-11,AAPL,C,2022-11-18,126.0,72.95,10.0,125.0,23.90,23.40,23.95,23.675
2,2022-11-11,AAPL,C,2022-11-18,127.0,72.75,14.0,265.0,22.70,22.40,23.00,22.700
3,2022-11-11,AAPL,C,2022-11-18,128.0,69.92,123.0,234.0,21.39,21.40,22.00,21.700
4,2022-11-11,AAPL,C,2022-11-18,129.0,69.43,46.0,230.0,20.35,20.40,21.05,20.725
...,...,...,...,...,...,...,...,...,...,...,...,...
59,2022-11-11,AAPL,P,2022-11-18,155.0,32.28,3315.0,32034.0,6.00,5.85,6.15,6.000
60,2022-11-11,AAPL,P,2022-11-18,157.5,37.65,25.0,1631.0,8.20,7.95,8.50,8.225
61,2022-11-11,AAPL,P,2022-11-18,160.0,39.31,1461.0,14584.0,10.60,10.20,10.75,10.475
62,2022-11-11,AAPL,P,2022-11-18,162.5,46.97,28.0,392.0,12.75,12.70,13.30,13.000


## Combining spot price and option query

### OHLC of all stocks

In [5]:
dfohlcall = getyahoodata(Opchain.assetlist)

[*********************100%***********************]  20 of 20 completed


D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
D:\Anaconda\lib\site-p

### Option chain with closing price >= 0.05 with greeks

In [12]:
asset1 = 'NVDA'
tradeday1 = '2022-10-13'
expiry1 = '2022-10-21'
opbound1 = 0.1
dfop1, info1 = Opchain.loaddayopchain(dfohlcall, tradeday=tradeday1, expiry=expiry1, asset=asset1, opbound=opbound1)

..\bsoption\nyopchain.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfcall['delta'] = dfcall['BS'].apply(lambda x: x.cdelta)
..\bsoption\nyopchain.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfcall['theta'] = dfcall['BS'].apply(lambda x: x.ctheta)
..\bsoption\nyopchain.py:139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [13]:
dfop1

,c_iv,c_vol,c_oi,c_mid,c_delta,c_theta,c_vega,c_gamma,p_iv,p_vol,p_oi,p_mid,p_delta,p_theta,p_vega,p_gamma
strike,,,,,,,,,,,,,,,,
95.0,92.19,139.0,125,24.875,0.960408,-0.087186,0.015132,0.005235,83.01,2353.0,4633,0.155,-0.026481,-0.056339,0.010859,0.004173
100.0,80.27,356.0,2003,19.975,0.941271,-0.104056,0.020741,0.008242,78.03,8205.0,11199,0.330,-0.054015,-0.094696,0.019417,0.007937
105.0,75.39,317.0,478,15.350,0.889198,-0.157697,0.033468,0.014160,73.83,12334.0,6179,0.700,-0.106425,-0.150021,0.032512,0.014046
110.0,70.31,5940.0,1084,11.025,0.803968,-0.215214,0.048975,0.022218,70.12,13305.0,8301,1.415,-0.195469,-0.214258,0.048889,0.022239
115.0,67.82,9114.0,1669,7.350,0.670331,-0.271694,0.064098,0.030146,66.97,10321.0,7921,2.695,-0.328104,-0.267775,0.063975,0.030470
117.0,67.09,2966.0,923,6.100,0.606784,-0.285520,0.068092,0.032373,65.63,2023.0,2259,3.400,-0.391740,-0.279014,0.068021,0.033058
118.0,66.38,3511.0,784,5.500,0.573850,-0.288025,0.069424,0.033359,64.94,1711.0,2016,3.800,-0.425376,-0.281672,0.069399,0.034086
119.0,65.21,2230.0,971,4.900,0.539973,-0.286449,0.070283,0.034378,64.50,1376.0,1277,4.250,-0.460008,-0.283328,0.070283,0.034756
120.0,64.99,14150.0,4899,4.400,0.505348,-0.286898,0.070632,0.034665,64.28,6488.0,8728,4.750,-0.495015,-0.283767,0.070633,0.035049


In [11]:
info1

('NVDA',
 datetime.datetime(2022, 10, 13, 0, 0),
 datetime.datetime(2022, 10, 21, 0, 0),
 119.6)

### Option chain with ITM calls only (cross check put-call parity)

In [14]:
asset2 = 'QQQ'
tradeday2 = '2022-11-04'
expiry2 = '2022-12-16'
opbound2 = 0.5
dfop2, info2 = Opchain.loaddayopchain(dfohlcall, tradeday=tradeday2, expiry=expiry2, asset=asset2, opbound=opbound2)
dfitmcall2 = dfop2.loc[:(info2[-1])]

..\bsoption\nyopchain.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfcall['delta'] = dfcall['BS'].apply(lambda x: x.cdelta)
..\bsoption\nyopchain.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfcall['theta'] = dfcall['BS'].apply(lambda x: x.ctheta)
..\bsoption\nyopchain.py:139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [15]:
dfitmcall2

,c_iv,c_vol,c_oi,c_mid,c_delta,c_theta,c_vega,c_gamma,p_iv,p_vol,p_oi,p_mid,p_delta,p_theta,p_vega,p_gamma
strike,,,,,,,,,,,,,,,,
210.0,46.56,18.0,15961,55.710,0.938727,-0.060264,0.108724,0.002897,39.36,3709.0,24016,0.510,-0.035931,-0.033215,0.070886,0.002234
215.0,44.52,136.0,845,50.915,0.926755,-0.066152,0.124816,0.003478,37.96,798.0,10999,0.665,-0.046595,-0.039551,0.087520,0.002860
216.0,44.61,1.0,100,49.980,0.922015,-0.069530,0.130925,0.003641,37.94,389.0,1356,0.715,-0.050133,-0.041913,0.092796,0.003034
217.0,44.34,1.0,90,49.055,0.918607,-0.071382,0.135230,0.003783,37.87,236.0,1441,0.765,-0.053655,-0.044154,0.097940,0.003208
218.0,43.85,9.0,84,48.100,0.916019,-0.072276,0.138454,0.003917,37.56,839.0,1311,0.815,-0.056307,-0.045495,0.101746,0.003360
219.0,43.26,1.0,94,47.125,0.913776,-0.072726,0.141215,0.004049,37.33,357.0,2072,0.860,-0.059450,-0.047189,0.106185,0.003529
220.0,43.03,7.0,1798,46.195,0.909825,-0.074796,0.146011,0.004209,37.07,667.0,19938,0.915,-0.062610,-0.048796,0.110571,0.003700
221.0,44.76,5.0,117,45.715,0.896847,-0.085882,0.161172,0.004467,36.96,781.0,1304,0.980,-0.066646,-0.051070,0.116069,0.003896
222.0,42.30,2.0,275,44.325,0.902725,-0.077759,0.154415,0.004528,36.66,1596.0,1510,1.040,-0.069945,-0.052580,0.120478,0.004077


In [34]:
asset3 = 'QQQ'
tradeday3 = '2022-07-14'
expiry3 = '2022-12-16'
opbound3 = 1
dfop3, info3 = Opchain.loaddayopchain(dfohlcall, tradeday=tradeday3, expiry=expiry3, asset=asset3, opbound=opbound3)
dfitmcall3 = dfop3.loc[:(info3[-1])]
dfitmcall3 = dfitmcall3[dfitmcall3['c_iv'] < 60]

..\bsoption\bsmodel.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  d_1 = (np.log(self.S / self.K) + self.T * (self.rf + 0.5 * (self.sig ** 2))) / (self.sig * (self.T ** 0.5))
..\bsoption\bsmodel.py:33: RuntimeWarning: invalid value encountered in double_scalars
  self.gamma = round(norm.pdf(self.d_1) / (self.S * self.sig * (self.T ** 0.5)), 9)
..\bsoption\nyopchain.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfcall['delta'] = dfcall['BS'].apply(lambda x: x.cdelta)
..\bsoption\nyopchain.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [35]:
dfitmcall3

,c_iv,c_vol,c_oi,c_mid,c_delta,c_theta,c_vega,c_gamma,p_iv,p_vol,p_oi,p_mid,p_delta,p_theta,p_vega,p_gamma
strike,,,,,,,,,,,,,,,,
165.0,56.65,1.0,97.0,124.020,0.953134,-0.033372,0.182619,0.000927,49.79,3.0,1013.0,1.055,-0.031501,-0.021220,0.132116,0.000763
170.0,55.26,1.0,230.0,119.250,0.948038,-0.035350,0.198306,0.001032,48.67,82.0,10110.0,1.320,-0.035895,-0.023095,0.147104,0.000869
175.0,53.93,4.0,109.0,114.505,0.942368,-0.037442,0.215225,0.001148,47.63,43.0,10909.0,1.510,-0.040972,-0.025176,0.163855,0.000989
180.0,52.44,9.0,692.0,109.740,0.936674,-0.039194,0.231694,0.001271,46.48,5.0,1230.0,1.615,-0.046288,-0.027110,0.180810,0.001119
185.0,51.22,1.0,397.0,105.055,0.929717,-0.041497,0.251156,0.001410,45.09,68.0,24422.0,1.920,-0.051406,-0.028598,0.196618,0.001254
190.0,50.03,4.0,700.0,100.400,0.922064,-0.043864,0.271792,0.001562,44.11,86.0,5952.0,2.190,-0.058396,-0.030943,0.217465,0.001418
195.0,54.38,1.0,148.0,95.790,0.896073,-0.059040,0.336565,0.001780,43.64,161.0,17262.0,2.560,-0.068000,-0.034468,0.244845,0.001613
200.0,50.65,1.0,28187.0,91.540,0.894411,-0.055626,0.340454,0.001933,42.24,187.0,53272.0,2.860,-0.075073,-0.035994,0.264163,0.001798
205.0,35.12,1.0,56.0,83.010,0.941996,-0.024506,0.216316,0.001771,41.90,23.0,1085.0,3.215,-0.087227,-0.039986,0.295841,0.002030
